<a href="https://colab.research.google.com/gist/ce12b078/d5409c87a51ab2b36ca8a2839c2c082d/content_based_recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plot Description Based Recommender

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/gdrive/My Drive"

 all_google_colab
 AnoushkaK_resume_fin.pdf
'BASIC STATISTICS AND PROBABILITY.pps'
 books
'Certificate for Printing'
'cfa level 1 notes (1).rar'
'cfa level 1 notes1.rar'
'cfa level 1 notes.rar'
'Cleaned data 2014.xlsx'
 code.R
'Colab Notebooks'
'comp list111.pdf'
'eri valuation.gsheet'
 google_colab
'home depot'
 HW4_report.pdf
'IAR intro PPT.gslides'
'Investment Analysis and Portfolio Management.pdf'
 kaggle
'Kreyszig - Advanced Engineering Mathematics.pdf'
 M13419521_mid_term_submission.pdf
 mycsv.csv
 mycsv.gsheet
'New DocPage 2'
'New Doc.pdf'
 nhai_proj1.pdf
 nhai_proj2_bot.pdf
'PPT TEMPLATE.gslides'
'(Prentice Hall finance series) James C. Van Horne-Financial management and policy-Prentice Hall (2002) (1).pdf'
 resume1111.pdf
'Resume Anshu Gupta.gdoc'
'Resume Anshu Gupta.pdf'
 roof_images.solitairetheme8
 TCS
'Technical Analysis_2.pdf'
 Test_final3.csv
'Time Series.pptx'
 Train_final3.csv
'U0009647_1009243989_INV (1).pdf'
 U0009647_1009243989_INV.pdf
'Untitled form (1).gform'
'Unt

In [0]:
import pandas as pd
df=pd.read_csv('gdrive/My Drive/all_google_colab/metadata_clean.csv')

In [0]:
import pandas as pd
import numpy as np

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [0]:
#Import the original file
orig_df = pd.read_csv('gdrive/My Drive/all_google_colab/movies_metadata.csv', low_memory=False)

#Add the useful features into the cleaned dataframe
df['overview'], df['id'] = orig_df['overview'], orig_df['id']

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [0]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [0]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
print(cosine_sim.shape)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [0]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [0]:
#Get recommendations for The Lion King
content_recommender('The Lion King')

# Metadata Based Recommender

In [0]:
# Load the keywords and credits files
cred_df = pd.read_csv('gdrive/My Drive/all_google_colab/credits.csv')
key_df = pd.read_csv('gdrive/My Drive/all_google_colab/keywords.csv')

In [0]:
#Print the head of the credit dataframe
cred_df.head()

In [0]:
#Print the head of the keywords dataframe
key_df.head()

In [0]:
type(df['id'][0])

In [0]:
#Convert the IDs of df into int
#df['id'] = df['id'].astype('int')

In [0]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [0]:
#Clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#Filter all rows that have a null ID
df = df[df['id'].notnull()]

In [0]:
# Convert IDs into integer
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')

#Display the head of df
df.head()

In [0]:
# Convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [0]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

In [0]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [0]:
#Define the new director feature
df['director'] = df['crew'].apply(get_director)

#Print the directors of the first five movies
df['director'].head()

In [0]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [0]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [0]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [0]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

In [0]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [0]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [0]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [0]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [0]:
#Display the soup of the first movie
df.iloc[0]['soup']

In [0]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [0]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [0]:
print(cosine_sim2.shape)

# Reset index of your df and construct reverse mapping again
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

In [0]:
content_recommender('The Lion King', cosine_sim2, df, indices2)

29607                                          Cheburashka
40904                   VeggieTales: Josh and the Big Wall
40913    VeggieTales: Minnesota Cuke and the Search for...
27768                                 The Little Matchgirl
15209             Spiderman: The Ultimate Villain Showdown
16613                            Cirque du Soleil: Varekai
24654                                  The Seventh Brother
29198                                      Superstar Goofy
30244                                              My Love
31179                Pokémon: Arceus and the Jewel of Life
Name: title, dtype: object

In [0]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [0]:
df.to_csv('hack_new.csv')
!cp hack_new.csv drive/My\ Drive/all_google_colab/